In [ ]:
import base
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rushd as rd
import scipy as sp
import seaborn as sns
from statannotations.Annotator import Annotator

# enables concurrent editing of base.py
from importlib import reload
reload(base)

### Load data

In [ ]:
base_path = rd.datadir/'instruments'/'data'/'attune'
metadata_path = rd.datadir/'projects'/'miR-iFFL'/'plasmids'
data, quantiles, stats, metadata = base.load_data(base_path, metadata_path, 'tuning', 'designs')

In [ ]:
# Create dicts to specify colors/markers
metadata_dict = metadata.set_index('construct').to_dict('dict')
main_palette = metadata_dict['color']
main_markers = metadata_dict['markers']

### Set up figure

In [ ]:
# Set plotting context
sns.set_style('ticks')
sns.set_context('paper', font_scale=1.0, rc=base.rc_context)
plt.rcParams.update(base.rc_params)
scatter_kwargs = base.scatter_kwargs
annotate_kwargs = base.annotate_kwargs

# Create the overall figure, gridspec, and add subfigure labels
fig = plt.figure(figsize=(base.figure_width['full'],4))
fig_gridspec = matplotlib.gridspec.GridSpec(2, 3, figure=fig, wspace=0.4, hspace=0.4, 
                                            height_ratios=[2.125,(4-2.125)], width_ratios=[1,1,1])

subfigures = {
    'A': fig.add_subfigure(fig_gridspec[0,0]),
    'B': fig.add_subfigure(fig_gridspec[0,1]),
    'C': fig.add_subfigure(fig_gridspec[0,2]),
    'D': fig.add_subfigure(fig_gridspec[1,0]),
    'E': fig.add_subfigure(fig_gridspec[1,1]),
    'F': fig.add_subfigure(fig_gridspec[1,2]),
}

# Add subpanel labels
for label, subfig in subfigures.items():
    if '2' in label: continue
    subfig.add_artist(matplotlib.text.Text(x=0, y=1, text=f'{label}', fontsize=base.font_sizes['subpanel_label'], 
                                           fontweight='bold', verticalalignment='top',transform=subfig.transSubfigure))

# Save to output folder
output_path = rd.rootdir/'output'/'fig_overview'/'fig_overview.pdf'
fig.savefig(rd.outfile(output_path))

In [ ]:
subfig = subfigures['E']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.5, bottom=0.4, top=0.35, right=0.15)
axes = subfig.subplots(1,2, gridspec_kw=dict(width_ratios=(1,0.3)))

biorep = 2
plot_df = quantiles[(((quantiles['miR']=='miR.FF5') & (quantiles['group']=='controller') & (quantiles['design']==1) & (quantiles['ts'].isin(['FF4x1','FF5x1','FF6x1']))) | (quantiles['group']=='base')) &
                         (quantiles['promoter']=='EF1a') & (quantiles['biorep']==biorep) & (quantiles['exp']!='exp90.4')]

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='construct', palette=main_palette, 
             legend=False, dashes=False, style='construct', markers=main_markers, ax=ax, markersize=4, markeredgewidth=0.5,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)))
ax.set(xscale='log', yscale='log', xlabel='marker', title='Initial circuit design', ylim=(2e1,1e5))
sns.despine(ax=ax)
marker_baseline = sp.stats.gmean(quantiles.loc[(quantiles['group']=='marker') & (quantiles['biorep']==biorep), 'output'])
ax.axhline(marker_baseline, color='black', ls=':')
ax.annotate('marker only', (ax.get_xlim()[1], marker_baseline), ha='right', va='bottom', fontsize=base.font_sizes['smaller_size'])

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df, y='output', hue='construct', palette=main_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax)
sns.despine(ax=ax, bottom=True)
ax.set(xlabel='', ylim=axes[0].get_ylim(), ylabel='', yticklabels=[])
ax.get_xaxis().set_visible(False)

for ax in axes: ax.minorticks_off()

fig.savefig(rd.outfile(output_path))

In [ ]:
subfig = subfigures['F']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.35, bottom=0.4, top=0.35, right=0.05)
axes = subfig.subplots(1,3, gridspec_kw=dict(width_ratios=(1,1,1), wspace=0.7))

# stat gmean
plot_df = stats[(((stats['miR']=='miR.FF5') & (stats['group']=='controller') & (stats['design']==1) & (stats['ts'].isin(['FF4x1','FF5x1','FF6x1']))) | (stats['group']=='base')) &
                         (stats['promoter']=='EF1a')]
xlim = (-0.5, len(plot_df['ts_kind'].unique())-0.5)
pairs = [('CL','base'), ('CL','OL')]

ax = axes[0]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='ts_label', y='output_gmean', hue='construct', palette=main_palette,
                  ax=ax, marker=main_markers[construct], **scatter_kwargs)
ax.set(title='Mean', xlim=xlim, xlabel='', ylabel='', yscale='log', ylim=(1e3,2e4),)

# stat std
ax = axes[1]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='ts_label', y='output_std', hue='construct', palette=main_palette,
                  ax=ax, marker=main_markers[construct], **scatter_kwargs)
ax.set(title='Std.', xlim=xlim, xlabel='', ylabel='', yscale='log', ylim=(1e3,4e4))

# slope
ax = axes[2]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='ts_label', y='slope', hue='construct', palette=main_palette,
                  ax=ax, marker=main_markers[construct], **scatter_kwargs)
ax.set(title='Slope', xlim=xlim, xlabel='', ylabel='', ylim=(0.4,1.3))

for ax in axes:
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right',)
    sns.despine(ax=ax)

fig.savefig(rd.outfile(output_path))

# perform statistical tests
f, axes = plt.subplots(1,3, figsize=(5,2))
pairs = [('base','CL'), ('OL','CL')]
stat_list = ['output_gmean', 'output_std', 'slope']
for ax, stat in zip(axes, stat_list):
    sns.stripplot(data=plot_df, x='ts_label', y=stat, ax=ax, hue='construct', palette=main_palette, legend=False,)
    if stat != 'slope': ax.set(yscale='log',)
    annotator = Annotator(ax, pairs, data=plot_df, x='ts_label', y=stat,)
    annotator.configure(**annotate_kwargs, line_offset=2).apply_and_annotate()

In [ ]:
# Save to OneDrive
fig.savefig(rd.outfile(rd.datadir/'manuscripts'/'2024_miR-iFFL'/'figures'/'links'/'fig_overview.pdf'))

In [ ]:
# Compare plotting styles

f, axes = plt.subplots(1,5, gridspec_kw=dict(wspace=0.7), figsize=(4,1.5))

plot_df = stats[(((stats['miR']=='miR.FF5') & (stats['group']=='controller') & (stats['design']==1) & (stats['ts'].isin(['FF4x1','FF5x1','FF6x1']))) | (stats['group']=='base')) &
                         (stats['promoter']=='EF1a')]
xlim = (-0.5, len(plot_df['ts_kind'].unique())-0.5)
pairs = [('CL','base'), ('CL','OL')]

# original style
ax = axes[0]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='ts_label', y='output_gmean', hue='construct', palette=main_palette,
                  ax=ax, marker=main_markers[construct], **scatter_kwargs)
ax.set(title='Original', xlim=xlim, xlabel='', ylabel='', yscale='log', ylim=(1e3,2e4),)

# with mean of bioreps as line
ax = axes[1]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='ts_label', y='output_gmean', hue='construct', palette=main_palette,
                  ax=ax, marker=main_markers[construct], **scatter_kwargs)
    
    sns.pointplot(data=group, x='ts_label', y='output_gmean', hue='construct', palette=main_palette,
                  ax=ax, linestyle='none', errorbar=None, marker='_', markersize=10, markeredgewidth=2,
                  legend=False,)
ax.set(title='Alt. A', xlim=xlim, xlabel='', ylabel='', yscale='log', ylim=(1e3,2e4),)

# single point with errorbar
ax = axes[2]
for construct, group in plot_df.groupby('construct'):
    sns.pointplot(data=group, x='ts_label', y='output_gmean', hue='construct', palette=main_palette,
                  ax=ax, linestyle='none', errorbar='se', marker=main_markers[construct], 
                  markersize=4, markeredgewidth=0.5, markeredgecolor='white', err_kws=dict(linewidth=1, zorder=0),
                  legend=False,)
ax.set(title='Alt. B', xlim=xlim, xlabel='', ylabel='', yscale='log', ylim=(1e3,2e4),)

# single point with errorbar, errorbar in front
ax = axes[3]
for construct, group in plot_df.groupby('construct'):
    sns.pointplot(data=group, x='ts_label', y='output_gmean', hue='construct', palette=main_palette,
                  ax=ax, linestyle='none', errorbar='se', marker=main_markers[construct], 
                  markersize=5, markeredgewidth=0.5, markeredgecolor='white', err_kws=dict(linewidth=1, zorder=100, color='black'),
                  legend=False,)
ax.set(title='Alt. C', xlim=xlim, xlabel='', ylabel='', yscale='log', ylim=(1e3,2e4),)

# single point with errorbar, no X marker
ax = axes[4]
for construct, group in plot_df.groupby('construct'):
    sns.pointplot(data=group, x='ts_label', y='output_gmean', hue='construct', palette=main_palette,
                  ax=ax, linestyle='none', errorbar='se', marker='o', 
                  markersize=5, markeredgewidth=0.5, markeredgecolor='white', err_kws=dict(linewidth=1, zorder=100, color='black'),
                  legend=False,)
ax.set(title='Alt. D', xlim=xlim, xlabel='', ylabel='', yscale='log', ylim=(1e3,2e4),)

for ax in axes:
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right',)

f.savefig(rd.outfile(rd.rootdir/'output'/'fig_overview'/'marker-styles.svg'), bbox_inches='tight')
f.savefig(rd.outfile(rd.rootdir/'output'/'fig_overview'/'marker-styles.png'), bbox_inches='tight')